In [1]:

"""
* @ Author: Lucas Glasner (lgvivanco96@gmail.com)
* @ Create Time: 1969-12-31 21:00:00
* @ Modified by: Lucas Glasner,
* @ Modified time: 2024-01-19 15:43:04
* @ Description:
        Routines for the design of  flood hydrographs based on synthetic
        hyetographs and unit hydrograph (SUH) methods.
* @ Dependencies:
*/
"""

import pandas as pd
import numpy as np
import scipy.signal as sg
import time
import tqdm

import xarray as xr
import os

from src.infiltration import CN_correction
from src.rain import design_storms, effective_storms
from src.unithydrographs import SynthUnitHydro
from src.misc import to_numeric

KeyboardInterrupt: 

In [ ]:
# ------------------------------- main routines ------------------------------ #
def parse_params(params_path, SHyeto_path, tstep=0.1):
    """
    Load input data and preprocess for the model run

    Args:
        params_path (str): Path to textfile with basin parameters. 
        SHyeto_path (str): Path to textfile with synthetic storm data-
        tstep (float, optional): Define simulation timestep. Defaults to 0.1.

    Returns:
        (tuple): formatted parameters,
                 daily rainfall, 
                 synthetic hyetographs,
                 timestep
    """    
    file_ext = params_path.split('.')[-1]
    if (file_ext == 'csv') or (file_ext == 'txt'):
        params = pd.read_csv(params_path, index_col=0)
    elif (file_ext == 'xlsx') or (file_ext == 'xls'):
        params = pd.read_excel(params_path, index_col=0)
    else:
        raise ValueError('Parameter extension file must be ".xlsx" or ".csv"')
    params = params.map(lambda x: to_numeric(x))
    
    # Prepare basin parameters
    # params.loc['final_curvenumber_1'] = [CN_correction(cn, amc)
    #                                     for cn, amc in zip(
    #                                         params.loc['curvenumber_1'],
    #                                         params.loc['moisturecondition_1'])]
    basin_params = params.loc[['name','area_km2', 'mriverlen_km',
                               'out2centroidlen_km', 'meanslope_1',
                               'hmax_m', 'hmin_m', 'SUnitHydro',
                               'curvenumber_1', 'moisturecondition_1']]
    cns = params.index.map(lambda index: 'fCN' in index)
    cns = params.loc[cns]
    basin_params = pd.concat([basin_params, cns], axis=0)


    # Prepare storm parameters
    pr_mm24hr = params[params.index.map(lambda x: 'pr' in x)]
    return_periods = pr_mm24hr.index.map(
        lambda x: int(x.split('_')[1].replace('T', '')))
    pr_mm24hr.index = return_periods
    params.loc['SHyeto'] = params.loc['SHyeto'].map(lambda x: x.split(';'))
    synth_hyeto = pd.read_csv(SHyeto_path, index_col=0)
    storm_params = {key:{'storm_durations':[4,6,12,18,24,30,36,42,48,60,72],
                         'synth_hyeto':synth_hyeto[params[key].loc['SHyeto']],
                         'rainfall':pr_mm24hr[key].dropna()}
                    for key in params.columns}

    return basin_params, storm_params, tstep

def BasinFloodHydrograph(timestep, basin_params, storm_params):
    """
    Main routine to compute a basin flood hydrograph from a set of 
    basin and storm parameters.

    Args:
        timestep (float): Simulation timestep in hours
        basin_params (dict): Basin parameters and geomorpholical properties.
        storm_params (dict): Storm parameters

    Returns:
        _type_: _description_
    """    
    # Compute Unit Hydrograph
    UH    = SynthUnitHydro(method=basin_params['SUnitHydro'],
                           basin_params=basin_params,
                           timestep=timestep)
    uh, uh_params = UH.compute()
    uhname        = uh.name.split('_')[0]
    tstep         = UH.timestep
    uh = uh.to_xarray().rename({'index':'time'})
    # uh = uh.reindex({'time':dhyeto.time})
    uh.attrs = {'standard_name':f'{uhname} unit hydrograph',
                'units':uh.name.split('_')[1],
                'qpeak':uh_params.qpeak.round(2),
                'tpeak':uh_params.tpeak.round(2),
                'tbase':uh_params.tbase.round(2),
                'tstep':uh_params.tstep.round(2)}
    uh.name = 'UnitHydro'
    
    # Compute design storms
    dhyeto  = design_storms(storm_duration=storm_params['storm_durations'],
                            synth_hyeto=storm_params['synth_hyeto'],
                            tstep=tstep,
                            precips=storm_params['rainfall']).astype(float)
    
    # dhyeto = effective_storms(dhyeto.pr, method='SCS',
    #                           CN=CN_correction(basin_params['curvenumber_1'],
    #                                            basin_params['moisturecondition_1']))
    cns   = basin_params.loc[basin_params.index.map(lambda index: 'fCN' in index)]
    farea = cns.values
    cns   = cns.index.map(lambda f: int(f.replace('fCN','').replace('_1','')))
    effstorms = []
    for cn, f in zip(cns, farea):
        cn = CN_correction(cn, basin_params.moisturecondition_1)
        x = effective_storms(dhyeto.pr, method='SCS', CN=cn)*f
        effstorms.append(x)
    effstorms = sum(effstorms)
    dhyeto  = effstorms.copy()
    
    # Compute flood hydrograph
    runoff = xr.apply_ufunc(sg.convolve, dhyeto.pr_eff.fillna(0), uh.fillna(0),
                input_core_dims=[['time'],['time']],
                output_core_dims=[['time'],],
                exclude_dims=set([('time'),]),
                keep_attrs=True,
                vectorize=True)
    # runoff = runoff.where(runoff>0)
    runoff.name = 'runoff'
    runoff.coords['time'] = runoff.time*tstep
    runoff.attrs = {'standard_name':'runoff', 'units':'m3*s1'}
    
    model = xr.merge([dhyeto, uh, runoff])
    model.attrs = basin_params.to_dict()
    model.attrs['timestep'] = tstep
    return model

In [ ]:
%%time
pd.set_option('future.no_silent_downcasting', True)
# ------------------------------ load parameters ----------------------------- #
basin_params, storm_params, tstep = parse_params('data/params.csv',
                                                 'data/synthethic_storms.csv')
basin_params = basin_params.T
basin_params['zone'] = ['I']*len(basin_params.index)
basin_params = basin_params.T
# Iterate over basins and run model
models = {key:None for key in basin_params.columns}
for fid in tqdm.tqdm(basin_params.columns, total=len(basin_params.columns)):
    name = basin_params[fid].loc['name'].replace(' ','')
    sim  = BasinFloodHydrograph(tstep, basin_params[fid], storm_params[fid])
    models[fid] = sim

  0%|          | 0/6 [00:00<?, ?it/s]

100%|██████████| 6/6 [00:03<00:00,  1.59it/s]

CPU times: total: 3.31 s
Wall time: 3.8 s


In [ ]:
for fid in models.keys():
    name = basin_params[fid].loc['name'].replace(' ','')
    if not os.path.isdir(f'data/Basins/{name}'):
        os.mkdir(f'data/Basins/{name}')
        os.mkdir(f'data/Basins/{name}/tmp')
        # time.sleep(0.1)
    
    models[fid].load().to_netcdf(f'data/Basins/{name}/tmp/SDH-Run_{name}.nc')